<a href="https://colab.research.google.com/github/ahakobia/Group4_NFLX_MIDTERM/blob/main/Adehs_Pyspark_US_Energy_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 2s (161 kB/s)
Reading package li

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

mkdir: cannot create directory ‘tests’: File exists
/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Overwriting __init__.py


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)

energy = energy.withColumnRenamed('_c0', 'id')
energy = energy.withColumnRenamed('YEAR', 'year')
energy = energy.withColumnRenamed('MONTH', 'month')
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.createOrReplaceTempView('energy_data')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy = p_energy.where("producer == 'Total Electric Power Industry' AND source == 'Total' AND state == 'US-TOTAL'")
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()

q_df = spark.sql("""SELECT year, (SUM(generated)) as Total_generated
            FROM p_energy_data 
            WHERE YEAR != 2022
            Group By YEAR
            Order By YEAR DESC
                """)

print("--- %s seconds ---" % (time.time() - start_time))

q_df.show(21)

--- 0.005460500717163086 seconds ---
+----+--------------------+
|year|     Total_generated|
+----+--------------------+
|2021|       4.115540153E9|
|2020|       4.007018594E9|
|2019|       4.126882145E9|
|2018|       4.178277345E9|
|2017|       4.034270559E9|
|2016|       4.076674986E9|
|2015|       4.077600939E9|
|2014|4.0929346911200004E9|
|2013|4.0659640674500003E9|
|2012|     4.04776525915E9|
|2011|     4.10065605012E9|
|2010|       4.125059899E9|
|2009|       3.950330927E9|
|2008|       4.119387759E9|
|2007|       4.156744725E9|
|2006|       4.064702227E9|
|2005|        4.05542275E9|
|2004|       3.970555264E9|
|2003|       3.883185204E9|
|2002|       3.858452251E9|
|2001|       3.736643649E9|
+----+--------------------+



In [68]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkEnergyData").getOrCreate()

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.createOrReplaceTempView('energy_data')
    energy.write.parquet('parquet_energy',mode='overwrite')
    return energy

def transform_data():
    p_energy = spark.read.parquet('parquet_energy')
    p_energy = p_energy.where("producer == 'Total Electric Power Industry' AND source == 'Total' AND state == 'US-TOTAL'")
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def query_data():
    q_df = spark.sql("""SELECT year, (SUM(generated)) as Total_generated
            FROM p_energy_data 
            WHERE YEAR != 2022
            Group By YEAR, State
            Order By YEAR DESC
                """)
    return q_df


Overwriting total_energy.py


In [69]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_energy.py

from total_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) == 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 21

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 2


Overwriting test_total_energy.py


In [70]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_energy.py ✓✓✓✓✓✓                                     100% ██████████

Results (23.57s):
       6 passed
